In [ ]:
# header / imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sc3nb as scn

In [ ]:
sc = scn.startup()

# Example for sc3nb.Buffer

* Buffer is a python class in the sc3nb package to interface with Buffers on the SuperCollider3 server.
* A buffer can be created with sc3nb.Buffer(sc)
* The constructor is also invoked by the method sc3nb.SC.Buffer(), a method of a booted SC instance, i.e. 
        buf = sc.Buffer()
  to allocate a buffer on the sound server booted from the SC instance.
* On buffer methods, the buffer instance is returned so that subsequent buffer methods (such as load_data, load_existing, etc.) can be directly invoked
* This notebook demonstrates with some examples how to work with sc3nb.Buffer

## Create Buffer from a numpy.Array

In [ ]:
d0 = np.random.rand(30000, 1)

In [ ]:
buf0 = sc.Buffer().load_data(d0)
buf0

## Buffer.query()

In this case a default buffer with default sample rate (44100) and default insert mode (ToDo: '...') is created. If you want to create a buffer with a specific sample rate or OSC insertion method etc.. 

Attention: the OSC insertion is only useful for small datasets (less than 1000 entries). For larger datasets the default file mode is much faster

In [ ]:
# uncomment following line to see help for the Buffer class:
# help(scn.Buffer)

In [ ]:
d0 = np.random.rand(30000, 1)
buf1 = sc.Buffer(bufnum=110).load_data(d0, sr=5000, mode="osc")
buf1

## Create Buffer with data from PyA Asig
This only works if using pya package: skip if not
* ToDo: check with multichannel signals

In [ ]:
from pya import *
a1 = Ugen().sine(15, dur=1.2, sr=1000).fade_out(0.5) # 0.5s sine tone of 100 Hz
buf1 = sc.Buffer(bufnum=120).load_asig(a1)

In [ ]:
print(a1, "\n", buf1)
a1.plot();

Again, default transport method is mode='file', i.e. using a temporary file and fill the buffer on sc
with this content. 
* use mode="osc" to select the direct transfer of data via OSC messages

## Create Buffer of .wav File

In [ ]:
buf2 = sc.Buffer().load_file("./my_recording.wav")
buf2

In [ ]:
buf2.play() # plays the buffer using a PlayBuf default synth, see below
print(buf2)
buf2.query()

The buffer method will automatically read the sample reate of the file and set it to Buffer.sr

## Allocate an empty Buffer

In [ ]:
buf3 = sc.Buffer().alloc(2.5*44100, sr=44100)
buf3

## Reuse an existing SC buffer
If you have already created a buffer in SC and want to have a sc3nb.Buffer object of that already existing buffer (identified by its bufnum), use Buffer.load_existing(bufnum):

In [ ]:
%sc b = Buffer.read(s, Platform.resourceDir +/+ "sounds/a11wlk01.wav");

In [ ]:
buf4num = %scg b.bufnum
buf4num

In [ ]:
buf4 = sc.Buffer().use_existing(buf4num)
buf4

In [ ]:
buf4.play(rate=2)

## Copy an existing SC buffer
If you want to copy an already existing buffer into another buffer, you can do it:

In [ ]:
buf2 = sc.Buffer().load_file("./my_recording.wav")
buf5 = sc.Buffer().copy_existing(buf2)

With this method, the complete buffer with all samples is copied. If you want to copy only a selection of samples, you can use `gen_copy()` (see below).

## Play Buffer

* The play method has the problem that different synths need to be used for different number of channels. 
    * The default play method creates the mono-channel synth 'pb-1ch'. 
    * For stereo buffers, manually overwrite by setting synth='pb-2ch'. 
* Future versions of Buffer should be able to determine the number of channels and automatically select an appropriate synth.

In [ ]:
d = np.sin(2*np.pi*440*np.linspace(0, 1, 44100)**0.9)
buf0 = sc.Buffer().load_data(d)

In [ ]:
buf0.play()  # play at rate 1

In [ ]:
buf0.play(rate=2, amp=0.05, pan=1 )  # play at given rate

In [ ]:
node_id = buf0.play(rate=4, loop=True)  # play looped 

In [ ]:
sc.msg("/n_free", node_id)  # free the buffer player 

## Write buffer content to file
Write the content of a buffer into a file. By default it is a .wav File with float as sample. You can change it via parameters "header" and "sample".

In [ ]:
buf0 = sc.Buffer().load_data(np.random.rand(10000)-0.5)

In [ ]:
buf0.write("./output.wav")

## Fetch buffer content to array

In [ ]:
# create a buffer
buf2 = sc.Buffer().load_file("./my_recording.wav")

In [ ]:
data = buf2.to_array()

In [ ]:
plt.plot(data);

In [ ]:
buf2.play(rate=1)

## Fill buffer with values

### Fill a buffer with zeros:

In [ ]:
help(scn.Buffer.zero)

In [ ]:
buf = sc.Buffer().alloc(100)
buf.zero()
plt.plot(buf.to_array());

### Fill a buffer range with values:

In [ ]:
help(scn.Buffer.fill)

In [ ]:
buf = sc.Buffer().alloc(500).fill(0, 90, 22).fill(200, 100, 5)
# ToDo: is not start, size, value but size, value 
plt.plot(buf.to_array());

In [ ]:
# BUG: wow: seems like the first start gets lost, the array is
# count, value, start, count, value, start, count, value, etc...
sc.msg("/b_fill", [buf._bufnum, [20, 50, -8000, 200, 100, 8000]])
# plt.plot(buf.to_array());
# TODO: @Micha: find out why first start gets lost

Fill buffer with multiple value triples:

In [ ]:
buf.fill([20, 50, -8000, 200, 100, 8000])
plt.plot(buf.to_array());

### Fill buffer with sinus waves and given amplitudes.

In [ ]:
help(scn.Buffer.gen_sine1)

In [ ]:
buf = sc.Buffer().alloc(500).gen_sine1([1,-0.5,0,1.4,0,0,0.2])
plt.plot(buf.to_array());

### Fill buffer with sinus waves and given frequency and amplitude

In [ ]:
help(scn.Buffer.gen_sine2)

In [ ]:
buf = sc.Buffer().alloc(1024).gen_sine2([[3.1, 1], [0.2, -2.5], [30, 0.3]])
plt.plot(buf.to_array());

### Fill buffer with sinus waves and given frequency, amplitude, phase

In [ ]:
help(scn.Buffer.gen_sine3)

In [ ]:
buf = sc.Buffer().alloc(1024).gen_sine3(
    [[1, 0.9, 1], [2, 0.3, +np.pi/2], [3, 0.3, 3]])
plt.plot(buf.to_array());

### Fill buffer with series of chebyshev polynomials:

In [ ]:
help(scn.Buffer.gen_cheby)

$\textrm{cheby}(n) = \textrm{amplitude} \cdot \cos(n \cdot \arccos(x))$

In [ ]:
buf = sc.Buffer().alloc(1024)
ch = [1]
for i in range(4):
    ch.insert(0, 0)
    buf.gen_cheby(ch)
    plt.plot(buf.to_array(), label=str(i));
plt.legend();

`gen_sine1` to `gen_sine3` and `gen_cheby` have the optional parameters:
* **normalize**: Normalize peak amplitude of wave to 1.0.
* **wavetable**: If set, then the buffer is written in wavetable format so that it can be read by interpolating oscillators.
* **clear**: if set then the buffer is cleared before new partials are written into it. Otherwise the new partials are summed with the existing contents of the buffer.

### Copy data of another buffer:

In [ ]:
help(scn.Buffer.gen_copy)

In [ ]:
buf1 = sc.Buffer().alloc(1024).fill(1024, 0, 0)
plt.plot(buf1.to_array());
buf2 = sc.Buffer().alloc(1024).gen_sine1([1,0.5,0,1.4,0,0.5,0.2])

# copy samples 0..0+400 of buf2 into buf1 at position 2++ 
buf1.gen_copy(buf2, 0, 2, 400)  
plt.plot(buf1.to_array());

# copy samples 250..end(=<0) of buf2 into buf1 at position 250++ 
buf1.gen_copy(buf2, 0, 250, 400)
plt.plot(buf1.to_array());

Here we copy 100 samples of `buf2` at starting pos 1 to buf3 at position 2. Use a negative amount of samples to copy all available samples

## Information about the buffer
Information about the buffer object:

In [ ]:
buf3

Information about the buffer in SC
(Known bug: You have to call this method multiple times, until you've got a list with the bufnum in the first list item)

In [ ]:
buf3.query()

## Delete & Free Buffers

* We start with a buffer

In [ ]:
buf = sc.Buffer().load_file("./my_recording.wav")
buf.play(synth="pb-2ch")

In [ ]:
print(buf)
buf.query()

### Delete Buffer in SC:

In [ ]:
buf.free()
# @Micha: TODO: reset _bufmode then...

In [ ]:
print(buf)  # listed as not loaded, python Buffer instance still exists

In [ ]:
# buf2.query()  # would give an error then...